<a href="https://colab.research.google.com/github/finloop/programowanie-liniowe/blob/main/Program-liniowy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programowanie liniowe z pomocą Python'a
 Ten notebook ma na celu pokazanie jak można rozwiązywać program liniowy z pomocą biblioteki `SciPy`. Zacznę od rozwiązania przykładowego zadania oraz pokażę w jaki sposób interpretować wyniki zwracene przez program. Następnie każdy kolejny przykład utrudnię zwiększając liczbę zmiennych oraz wprowadzając sprzeczności.

# Rozwiązywanie problemu w postaci kanonicznej
Z dokumentacji biblioteki `SciPy` odczytać możemy w jakiej postaci przyjmuje ona problemy w postaci: $$\min_x \ c^T x \\  
A_{ub} x \leq b_{ub},\\ 
A_{eq} x = b_{eq},\\ 
l \leq x \leq u$$
Gdzie:
*   $\min_x \ c^T x$ to funkcja celu
*   $x$ to wektor zmiennych decyzyjnych
*   $c^T$ to macierz transponowana do macierzy jednokolumnowej C tzn. $c^T = [c_1, \dots , c_n]$
*   $A_{ub}$ - wiersze macierzy ograniczeń dla, których ograniczenie jest w postaci $\leq$
*   $b_{ub}$ - ograniczenia górne dla wierszy $A_{ub}$
*   $A_{eq}$ - wiersze macierzy ograniczeń dla, których ograniczenie jest w postaci $ = $
*   $b_{eq}$ - ograniczenia górne dla wierszy $A_{eq}$
*   $l$ dolne ograniczenie dla $x$ domyślnie $0$

Dla zadania w postaci kanonicznej będziemy korzystać wyłącznie z części:
$$\min_x \ c^T x \\  
A_{eq} x = b_{eq},\\ 
0 \leq x $$




In [ ]:
from scipy.optimize import linprog